# Étude de marché

*La poule qui chante, une entreprise française d’agroalimentaire souhaite se développer à l'international. L'international, oui, mais pour l'instant, le champ des possibles est bien large : aucun pays particulier ni aucun continent n'est pour le moment choisi. Tous les pays sont envisageables !*

**Objectif de cette étude :** Obtenir une liste de pays dans lesquels la société pourrait potentiellement s'exporter

Dans cette première partie, nous allons importer des données recueillies sur le site de la FAO afin de constituer le fichier support que nous analyserons par la suite.

In [1]:
# Import des librairies et des fonctions nécessaires

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels as sm
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


### Chargement et nettoyage des données relatives aux disponibilités alimentaires

In [2]:
# Import et visualisation des fichiers transmis

dispo_alim = pd.read_csv('DisponibiliteAlimentaire_2017.csv')
dispo_alim.head()

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole
0,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5511,Production,2511,Blé et produits,2017,2017,Milliers de tonnes,4281.0,S,Données standardisées
1,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5611,Importations - Quantité,2511,Blé et produits,2017,2017,Milliers de tonnes,2302.0,S,Données standardisées
2,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5072,Variation de stock,2511,Blé et produits,2017,2017,Milliers de tonnes,-119.0,S,Données standardisées
3,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5911,Exportations - Quantité,2511,Blé et produits,2017,2017,Milliers de tonnes,0.0,S,Données standardisées
4,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5301,Disponibilité intérieure,2511,Blé et produits,2017,2017,Milliers de tonnes,6701.0,S,Données standardisées


In [3]:
# Recherche d'éventuels doublons et/ou valeurs manquantes dans dispo_alim

print('Doublons présents dans le fichier des disponbilités alimentaires : ' + str(dispo_alim.duplicated().sum()))
print('Valeurs manquantes dans le fichier des disponbilités alimentaires : ' + str(dispo_alim.isna().sum()))

Doublons présents dans le fichier des disponbilités alimentaires : 0
Valeurs manquantes dans le fichier des disponbilités alimentaires : Code Domaine              0
Domaine                   0
Code zone                 0
Zone                      0
Code Élément              0
Élément                   0
Code Produit              0
Produit                   0
Code année                0
Année                     0
Unité                     0
Valeur                    0
Symbole                   0
Description du Symbole    0
dtype: int64


In [4]:
# Affichage d'un résumé des valeurs présentes dans dispo_alim

dispo_alim['Valeur'].describe()

count    176600.000000
mean        210.554918
std        4762.047787
min      -10388.000000
25%           0.000000
50%           0.120000
75%           8.462500
max      758548.000000
Name: Valeur, dtype: float64

La valeur minimale dans les disponibilités alimentaires étant négative, vérifions ce qu'elle représente avant de prendre une quelconque décision la concernant.

In [5]:
dispo_alim[dispo_alim['Valeur'] == (-10388.000000)]

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole
77209,FBS,Nouveaux Bilans Alimentaire,101,Indonésie,5072,Variation de stock,2562,Palmistes,2017,2017,Milliers de tonnes,-10388.0,S,Données standardisées


Il n'est pas absurde de rencontrer des valeurs négatives relatives à la variation des stocks donc il n'est pas nécessaire de faire quoi que ce soit d'autant plus que cette variable ne sera pas utile à l'analyse, comme un certains nombre d'autres variables disponibles.

In [6]:
# Visualisation des colonnes du fichier

print(dispo_alim.columns)

Index(['Code Domaine', 'Domaine', 'Code zone', 'Zone', 'Code Élément',
       'Élément', 'Code Produit', 'Produit', 'Code année', 'Année', 'Unité',
       'Valeur', 'Symbole', 'Description du Symbole'],
      dtype='object')


Certaines de ces colonnes ne nous seront d'aucune utilité, nous pouvons donc les supprimer.

In [7]:
# Suppression des colonnes non pertinentes

dispo_alim.drop(['Code Domaine', 'Domaine', 'Code zone', 'Code Élément', 'Code Produit', 'Code année', 'Symbole', 'Description du Symbole'],  axis=1, inplace=True)
dispo_alim.head()

,Zone,Élément,Produit,Année,Unité,Valeur
0,Afghanistan,Production,Blé et produits,2017,Milliers de tonnes,4281.0
1,Afghanistan,Importations - Quantité,Blé et produits,2017,Milliers de tonnes,2302.0
2,Afghanistan,Variation de stock,Blé et produits,2017,Milliers de tonnes,-119.0
3,Afghanistan,Exportations - Quantité,Blé et produits,2017,Milliers de tonnes,0.0
4,Afghanistan,Disponibilité intérieure,Blé et produits,2017,Milliers de tonnes,6701.0


Jetons également un oeil sur les différents éléments mis à notre disposition afin de ne sélectionner, par la suite, que ceux qui nous seront utiles.

In [8]:
# Affichage des différents éléments de dispo_alim 

print(dispo_alim['Élément'].unique())

['Production' 'Importations - Quantité' 'Variation de stock'
 'Exportations - Quantité' 'Disponibilité intérieure'
 'Aliments pour animaux' 'Semences' 'Pertes' 'Résidus' 'Nourriture'
 'Disponibilité alimentaire en quantité (kg/personne/an)'
 'Disponibilité alimentaire (Kcal/personne/jour)'
 'Disponibilité de protéines en quantité (g/personne/jour)'
 'Disponibilité de matière grasse en quantité (g/personne/jour)'
 'Traitement' 'Autres utilisations (non alimentaire)'
 'Alimentation pour touristes']


Toujours dans le but de faciliter l'étude des disponibilités alimentaires, ajoutons l'origine (animale ou végétale) des produits disponibles à notre base de données.

In [9]:
# Import du fichier contenant les produits ainsi que leurs origines

produits = pd.read_csv('Produit.csv', sep=';')
produits.head()

,Produit,Origine
0,Abats Comestible,Animale
1,"Beurre, Ghee",Animale
2,Crème,Animale
3,Graisses Animales Crue,Animale
4,Lait - Excl Beurre,Animale


In [10]:
# Jointure entre dispo_alim et produits

dispo_alim = pd.merge(dispo_alim, produits, on='Produit')
dispo_alim.head()

,Zone,Élément,Produit,Année,Unité,Valeur,Origine
0,Afghanistan,Production,Blé et produits,2017,Milliers de tonnes,4281.0,Végétale
1,Afghanistan,Importations - Quantité,Blé et produits,2017,Milliers de tonnes,2302.0,Végétale
2,Afghanistan,Variation de stock,Blé et produits,2017,Milliers de tonnes,-119.0,Végétale
3,Afghanistan,Exportations - Quantité,Blé et produits,2017,Milliers de tonnes,0.0,Végétale
4,Afghanistan,Disponibilité intérieure,Blé et produits,2017,Milliers de tonnes,6701.0,Végétale


Modifions la forme de notre table en transposant chacun des éléments dans des colonnes afin d'en faciliter par la suite l'exploitation et supprimons les éléments inutiles à l'analyse.

In [11]:
# Pivot de la table dispo_alim

dispo_alim = dispo_alim.pivot_table(index=['Zone', 'Origine'], columns='Élément', values='Valeur', aggfunc=sum).reset_index()
dispo_alim.head()

Élément,Zone,Origine,Alimentation pour touristes,Aliments pour animaux,Autres utilisations (non alimentaire),Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Pertes,Production,Résidus,Semences,Traitement,Variation de stock
0,Afghanistan,Animale,NaN,165.0,0.0,190.0,67.42,13.26,10.56,2707.0,0.0,170.0,2447.0,91.0,2538.0,0.0,5.0,NaN,0.0
1,Afghanistan,Végétale,NaN,291.0,94.0,1807.0,289.89,17.37,43.53,12432.0,601.0,4385.0,10521.0,1055.0,9071.0,-111.0,465.0,115.0,426.0
2,Afrique du Sud,Animale,0.0,220.0,106.0,454.0,131.80,32.41,34.56,8180.0,1192.0,1247.0,7515.0,357.0,8119.0,-80.0,60.0,0.0,-4.0
3,Afrique du Sud,Végétale,0.0,9151.0,1342.0,2533.0,424.62,49.51,48.80,58660.0,9776.0,7773.0,24211.0,2751.0,60336.0,-514.0,152.0,21564.0,-318.0
4,Albanie,Animale,NaN,96.0,0.0,1094.0,473.16,75.29,66.32,1412.0,10.0,102.0,1363.0,66.0,1326.0,-112.0,0.0,0.0,5.0


In [12]:
# Suppression des colonnes inutiles à l'analyse

dispo_alim.drop(['Alimentation pour touristes', 'Aliments pour animaux', 'Autres utilisations (non alimentaire)','Disponibilité alimentaire en quantité (kg/personne/an)', 'Disponibilité de matière grasse en quantité (g/personne/jour)',
                'Disponibilité intérieure', 'Exportations - Quantité', 'Importations - Quantité', 'Nourriture',
                'Pertes', 'Production', 'Semences', 'Traitement', 'Résidus','Variation de stock'], axis=1, inplace=True)
dispo_alim.head()

Élément,Zone,Origine,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité de protéines en quantité (g/personne/jour)
0,Afghanistan,Animale,190.0,10.56
1,Afghanistan,Végétale,1807.0,43.53
2,Afrique du Sud,Animale,454.0,34.56
3,Afrique du Sud,Végétale,2533.0,48.80
4,Albanie,Animale,1094.0,66.32


Le but de cette analyse étant d'exporter des poulets, focalisons nous sur les produits d'origine animale et notamment sur la proportion de la disponibilité de protéines d'origine animale par rapport à la disponibilité totale en protéines.

In [13]:
# Création d'un dataframe axé sur les disponibilités animales

dispo_an = dispo_alim[dispo_alim['Origine'] == 'Animale'].reset_index(drop=True)
del dispo_an['Origine']
dispo_an.head()

Élément,Zone,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité de protéines en quantité (g/personne/jour)
0,Afghanistan,190.0,10.56
1,Afrique du Sud,454.0,34.56
2,Albanie,1094.0,66.32
3,Algérie,402.0,25.70
4,Allemagne,1092.0,63.08


In [14]:
# Création d'un dataframe groupant toutes les disponibilités alimentaires par pays

dispo_pays = dispo_alim.groupby('Zone').sum().reset_index()
dispo_pays.head()

Élément,Zone,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité de protéines en quantité (g/personne/jour)
0,Afghanistan,1997.0,54.09
1,Afrique du Sud,2987.0,83.36
2,Albanie,3400.0,119.50
3,Algérie,3345.0,92.85
4,Allemagne,3559.0,104.07


In [15]:
# Calcul de la proportion de la disponibilité de protéines d'origine animale par rapport à la disponibilité totale

dispo_pays['Proportion protéines animales (%)'] = round(dispo_an['Disponibilité de protéines en quantité (g/personne/jour)'] / dispo_pays['Disponibilité de protéines en quantité (g/personne/jour)']*100, 2)
dispo_pays.head()

Élément,Zone,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Proportion protéines animales (%)
0,Afghanistan,1997.0,54.09,19.52
1,Afrique du Sud,2987.0,83.36,41.46
2,Albanie,3400.0,119.50,55.50
3,Algérie,3345.0,92.85,27.68
4,Allemagne,3559.0,104.07,60.61


### Chargement et nettoyage des données relatives aux populations

In [16]:
# Import du fichier population

population = pd.read_csv('Population_2000_2018.csv')
population.head()

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole,Note
0,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2000,2000,1000 personnes,20779.953,X,Sources internationales sûres,NaN
1,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2001,2001,1000 personnes,21606.988,X,Sources internationales sûres,NaN
2,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2002,2002,1000 personnes,22600.770,X,Sources internationales sûres,NaN
3,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2003,2003,1000 personnes,23680.871,X,Sources internationales sûres,NaN
4,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2004,2004,1000 personnes,24726.684,X,Sources internationales sûres,NaN


In [17]:
# Recherche d'éventuels doublons et/ou valeurs manquantes dans population

print('Doublons présents dans le fichier population : ' + str(population.duplicated().sum()))
print('Valeurs manquantes dans le fichier population : ' + str(population.isna().sum()))

Doublons présents dans le fichier population : 0
Valeurs manquantes dans le fichier population : Code Domaine                 0
Domaine                      0
Code zone                    0
Zone                         0
Code Élément                 0
Élément                      0
Code Produit                 0
Produit                      0
Code année                   0
Année                        0
Unité                        0
Valeur                       0
Symbole                      0
Description du Symbole       0
Note                      4153
dtype: int64


In [18]:
# Affichage d'un résumé des valeurs présentes dans population

population['Valeur'].describe()

count    4.411000e+03
mean     2.963004e+04
std      1.238029e+05
min      7.850000e-01
25%      3.921890e+02
50%      4.764741e+03
75%      1.821548e+04
max      1.427648e+06
Name: Valeur, dtype: float64

In [19]:
# Affichage des différentes colonnes de population

print(population.columns)

Index(['Code Domaine', 'Domaine', 'Code zone', 'Zone', 'Code Élément',
       'Élément', 'Code Produit', 'Produit', 'Code année', 'Année', 'Unité',
       'Valeur', 'Symbole', 'Description du Symbole', 'Note'],
      dtype='object')


Encore une fois nous allons supprimer les colonnes qui ne nous seront d'aucune utilité.

In [20]:
# Suppression desdites colonnes 

population.drop(['Code Domaine', 'Domaine', 'Code zone', 'Code Élément', 'Élément', 'Code Produit', 'Produit', 'Code année', 'Unité', 'Symbole', 'Description du Symbole', 'Note'], axis=1, inplace=True)
population['Valeur'] = population['Valeur']*1000
population.head()

,Zone,Année,Valeur
0,Afghanistan,2000,20779953.0
1,Afghanistan,2001,21606988.0
2,Afghanistan,2002,22600770.0
3,Afghanistan,2003,23680871.0
4,Afghanistan,2004,24726684.0


Les données relatives aux disponibiités alimentaires en notre possession correspondant à l'année 2017, nous avons pris le parti de centrer notre étude sur 2017 et les 4 années précédentes en calculant le taux moyen d'évolution de la population.

In [21]:
# Affichage des différentes années disponibles dans population

population['Année'].describe

<bound method NDFrame.describe of 0       2000
1       2001
2       2002
3       2003
4       2004
        ... 
4406    2014
4407    2015
4408    2016
4409    2017
4410    2018
Name: Année, Length: 4411, dtype: int64>

In [22]:
# Création de dataframes regroupant les données de population de 2013 à 2017

pop_2013 = population[population['Année'] == 2013]
del pop_2013['Année']

pop_2014 = population[population['Année'] == 2014]
del pop_2014['Année']

pop_2015 = population[population['Année'] == 2015]
del pop_2015['Année']

pop_2016 = population[population['Année'] == 2016]
del pop_2016['Année']
                      
pop_2017 = population[population['Année'] == 2017]
del pop_2017['Année']

In [23]:
# Jointure des différentes tables de population

population_1314 = pd.merge(pop_2013, pop_2014, on='Zone')
population_1314.rename(columns={'Valeur_x': 'Population 2013', 
                                'Valeur_y': 'Population 2014'}, inplace=True)

population_1315 = pd.merge(population_1314, pop_2015, on='Zone')
population_1315.rename(columns={'Valeur': 'Population 2015'}, inplace=True)

population_1316 = pd.merge(population_1315, pop_2016, on='Zone')
population_1316.rename(columns={'Valeur': 'Population 2016'}, inplace=True)

population_1317 = pd.merge(population_1316, pop_2017, on='Zone')
population_1317.rename(columns={'Valeur': 'Population 2017'}, inplace=True)
population_1317.head()

,Zone,Population 2013,Population 2014,Population 2015,Population 2016,Population 2017
0,Afghanistan,32269589.0,33370794.0,34413603.0,35383032.0,36296113.0
1,Afrique du Sud,53687121.0,54544186.0,55386367.0,56207646.0,57009756.0
2,Albanie,2903790.0,2896305.0,2890513.0,2886438.0,2884169.0
3,Algérie,38140133.0,38923692.0,39728025.0,40551392.0,41389189.0
4,Allemagne,81174367.0,81450378.0,81787411.0,82193768.0,82658409.0


In [24]:
# Calcul des taux d'évolution de la population

population_1317['Taux évolution population 2013/2014 (%)'] = round(((population_1317['Population 2014'] - population_1317['Population 2013']))/population_1317['Population 2013']*100, 2)
population_1317['Taux évolution population 2014/2015 (%)'] = round(((population_1317['Population 2015'] - population_1317['Population 2014']))/population_1317['Population 2014']*100, 2)
population_1317['Taux évolution population 2015/2016 (%)'] = round(((population_1317['Population 2016'] - population_1317['Population 2015']))/population_1317['Population 2015']*100, 2)
population_1317['Taux évolution population 2016/2017 (%)'] = round(((population_1317['Population 2017'] - population_1317['Population 2016']))/population_1317['Population 2016']*100, 2)
population_1317.head()

,Zone,Population 2013,Population 2014,Population 2015,Population 2016,Population 2017,Taux évolution population 2013/2014 (%),Taux évolution population 2014/2015 (%),Taux évolution population 2015/2016 (%),Taux évolution population 2016/2017 (%)
0,Afghanistan,32269589.0,33370794.0,34413603.0,35383032.0,36296113.0,3.41,3.12,2.82,2.58
1,Afrique du Sud,53687121.0,54544186.0,55386367.0,56207646.0,57009756.0,1.60,1.54,1.48,1.43
2,Albanie,2903790.0,2896305.0,2890513.0,2886438.0,2884169.0,-0.26,-0.20,-0.14,-0.08
3,Algérie,38140133.0,38923692.0,39728025.0,40551392.0,41389189.0,2.05,2.07,2.07,2.07
4,Allemagne,81174367.0,81450378.0,81787411.0,82193768.0,82658409.0,0.34,0.41,0.50,0.57


In [25]:
# Calcul du taux d'évolution moyen de la population par pays sur les 5 dernières années

population_1317['Taux évolution moyen population (%)'] = round((population_1317['Taux évolution population 2013/2014 (%)'] + population_1317['Taux évolution population 2014/2015 (%)'] + population_1317['Taux évolution population 2015/2016 (%)'] + population_1317['Taux évolution population 2016/2017 (%)'])/4, 2)

In [26]:
# Création d'un nouveau dataframe contenant les pays et leurs taux moyen d'évolution de la population

pop_evol = population_1317
pop_evol.drop(['Population 2013', 'Population 2014', 'Population 2015', 'Population 2016', 'Population 2017', 'Taux évolution population 2013/2014 (%)', 'Taux évolution population 2014/2015 (%)', 'Taux évolution population 2015/2016 (%)', 'Taux évolution population 2016/2017 (%)'], axis=1, inplace=True)
pop_evol.head()

,Zone,Taux évolution moyen population (%)
0,Afghanistan,2.98
1,Afrique du Sud,1.51
2,Albanie,-0.17
3,Algérie,2.06
4,Allemagne,0.45


### Chargement des données relatives aux PIBs

Encore une fois, nous allons calculer le PIB moyen par habitant entre les années 2013 et 2017, et ne conserver que les colonnes utiles à notre analyse.

In [27]:
# Import du fichier PIB téléchargé sur le site de la FAO (https://www.fao.org/faostat/fr/#data/MK)

pib = pd.read_csv('PIB.csv')

# Sélection des années 2013 à 2017

pib_1317 = pib[(pib['Année'] == 2013) + (pib['Année'] == 2014) + (pib['Année'] == 2015) + (pib['Année'] == 2016) + (pib['Année'] == 2017)]
pib_1317.head()

,Code Domaine,Domaine,Code zone (M49),Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole,Note
26,MK,Indicateurs macro,4,Afghanistan,6110,Valeur US $,22008,Produit Intérieur Brut,2013,2013,millions,19621.802455,X,Ciffre de sources internationales,NaN
27,MK,Indicateurs macro,4,Afghanistan,6119,Valeur US $ par habitant,22008,Produit Intérieur Brut,2013,2013,US$,608.058642,X,Ciffre de sources internationales,NaN
28,MK,Indicateurs macro,4,Afghanistan,6110,Valeur US $,22008,Produit Intérieur Brut,2014,2014,millions,19550.702567,X,Ciffre de sources internationales,NaN
29,MK,Indicateurs macro,4,Afghanistan,6119,Valeur US $ par habitant,22008,Produit Intérieur Brut,2014,2014,US$,585.862673,X,Ciffre de sources internationales,NaN
30,MK,Indicateurs macro,4,Afghanistan,6110,Valeur US $,22008,Produit Intérieur Brut,2015,2015,millions,18713.048410,X,Ciffre de sources internationales,NaN


In [28]:
# Sélection des colonnes utiles à l'analyse

pib_1317 = pib_1317[['Zone', 'Année', 'Élément', 'Valeur']]
pib_1317.head()

# Mise en forme du dataframe

pib_1317 = pib_1317[pib_1317['Élément'] == 'Valeur US $ par habitant']
del pib_1317['Élément']
pib_1317.head()

,Zone,Année,Valeur
27,Afghanistan,2013,608.058642
29,Afghanistan,2014,585.862673
31,Afghanistan,2015,543.768940
33,Afghanistan,2016,509.271172
35,Afghanistan,2017,520.616409


In [29]:
# Agrégation des PIBs/habitant en moyenne

pib_1317 = pib_1317.groupby('Zone').mean()
del pib_1317['Année']
pib_1317['Valeur'] = round(pib_1317['Valeur'], 2)
pib_1317.rename(columns={'Valeur': 'PIB moyen/habitant ($)'}, inplace=True)
pib_1317.head()

,PIB moyen/habitant ($)
Zone,
Afghanistan,553.52
Afrique du Sud,6076.57
Albanie,4306.02
Algérie,4645.34
Allemagne,44308.71


Nous allons maintenant joindre les dataframes contenant les données relatives aux populations et aux PIBs.

In [30]:
# Jointure de pib_1317 et pop_evol

pop_evol = pd.merge(pop_evol, pib_1317, on='Zone')
pop_evol.head()

,Zone,Taux évolution moyen population (%),PIB moyen/habitant ($)
0,Afghanistan,2.98,553.52
1,Afrique du Sud,1.51,6076.57
2,Albanie,-0.17,4306.02
3,Algérie,2.06,4645.34
4,Allemagne,0.45,44308.71


Nous allons maintenant joindre les disponibilités aux populations mais vérifions d'abord les tailles des 2 dataframes.

In [31]:
# Vérification des tailles de pop_evol et dispo_pays

print(pop_evol.shape)
print(dispo_pays.shape)

(209, 3)
(174, 4)


Prenons le parti de choisir d'effectuer une jointure interne afin de limiter le risque de nous retrouver avec des valeurs manquantes.

In [32]:
# Jointure interne entre dispo_pays et pop_evol

dispo_pop = pd.merge(dispo_pays, pop_evol, on='Zone')
dispo_pop.head()

,Zone,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Proportion protéines animales (%),Taux évolution moyen population (%),PIB moyen/habitant ($)
0,Afghanistan,1997.0,54.09,19.52,2.98,553.52
1,Afrique du Sud,2987.0,83.36,41.46,1.51,6076.57
2,Albanie,3400.0,119.50,55.50,-0.17,4306.02
3,Algérie,3345.0,92.85,27.68,2.06,4645.34
4,Allemagne,3559.0,104.07,60.61,0.45,44308.71


In [33]:
# Quid de la taille de dispo_pop

dispo_pop.shape

(173, 6)

Intéressons nous également aux quantités de poulets importées par les pays afin de mieux les cibler.

### Chargement des données relatives aux importations

Encore une fois, calculons les valeurs moyennes des importations entre les années 2013 et 2017, et ne conserver que les colonnes utiles à notre analyse.

In [34]:
# Importation des données téléchargées sur le site de la FAO (https://www.fao.org/faostat/fr/#data/TCL)

importation = pd.read_csv('Importations.csv')
importation.head()

,Code Domaine,Domaine,Code zone (M49),Zone,Code Élément,Élément,Code Produit (CPC),Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole
0,TCL,Cultures et produits animaux,4,Afghanistan,5622,Importations - Valeur,2151,Poulets,2013,2013,1000 US$,3415.0,T,Chiffre non officiel
1,TCL,Cultures et produits animaux,4,Afghanistan,5622,Importations - Valeur,2151,Poulets,2014,2014,1000 US$,5748.0,T,Chiffre non officiel
2,TCL,Cultures et produits animaux,4,Afghanistan,5622,Importations - Valeur,2151,Poulets,2015,2015,1000 US$,2330.0,T,Chiffre non officiel
3,TCL,Cultures et produits animaux,4,Afghanistan,5622,Importations - Valeur,2151,Poulets,2016,2016,1000 US$,7956.0,T,Chiffre non officiel
4,TCL,Cultures et produits animaux,4,Afghanistan,5622,Importations - Valeur,2151,Poulets,2017,2017,1000 US$,26227.0,A,Chiffre officiel


In [35]:
# Suppression des colonnes inutiles à l'analyse, à savoir toutes exceptées "Zone" et "Valeur" (après avoir multiplié les valeurs par 1000)

importation['Valeur'] = (importation['Valeur']*1000)
importation.drop(['Code Domaine', 'Domaine', 'Code zone (M49)', 'Code Élément', 'Élément', 'Code Produit (CPC)', 'Produit', 'Code année', 'Année', 'Unité', 'Symbole', 'Description du Symbole'], axis=1, inplace=True)
importation = importation.groupby('Zone').mean()
importation.head()

,Valeur
Zone,
Afghanistan,9135200.0
Afrique du Sud,7425600.0
Albanie,5066400.0
Algérie,21107000.0
Allemagne,173673000.0


In [36]:
# Renommage de la colonne "Valeur"

importation.rename(columns={'Valeur': 'Quantité moyenne poulets importée ($)'}, inplace=True)
importation.head()

,Quantité moyenne poulets importée ($)
Zone,
Afghanistan,9135200.0
Afrique du Sud,7425600.0
Albanie,5066400.0
Algérie,21107000.0
Allemagne,173673000.0


Joignons maintenant les importations aux disponibilités en effectuant encore une fois une jointure interne afin d'obtenir notre dataframe global.

In [37]:
# Jointure interne entre dispo_pop et importation

dispo_pop = pd.merge(dispo_pop, importation, on='Zone')
dispo_pop.head()

,Zone,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Proportion protéines animales (%),Taux évolution moyen population (%),PIB moyen/habitant ($),Quantité moyenne poulets importée ($)
0,Afghanistan,1997.0,54.09,19.52,2.98,553.52,9135200.0
1,Afrique du Sud,2987.0,83.36,41.46,1.51,6076.57,7425600.0
2,Albanie,3400.0,119.50,55.50,-0.17,4306.02,5066400.0
3,Algérie,3345.0,92.85,27.68,2.06,4645.34,21107000.0
4,Allemagne,3559.0,104.07,60.61,0.45,44308.71,173673000.0


In [38]:
dispo_pop.shape

(169, 7)

### Nettoyage de notre dataframe global

In [39]:
# Recherche de doublons dans le dataframe

print('Doublons présents dans le dataframe : ' + str(dispo_pop.duplicated().sum()))

Doublons présents dans le dataframe : 0


In [40]:
# Recherche d'éventuelles valeurs manquantes dans les fichiers

print('Valeurs manquantes dans le dataframe : ' + str(dispo_pop.isna().sum()))

Valeurs manquantes dans le dataframe : Zone                                                        0
Disponibilité alimentaire (Kcal/personne/jour)              0
Disponibilité de protéines en quantité (g/personne/jour)    0
Proportion protéines animales (%)                           1
Taux évolution moyen population (%)                         0
PIB moyen/habitant ($)                                      0
Quantité moyenne poulets importée ($)                       1
dtype: int64


Nous avons des valeurs manquantes sur certaines variables, vérifions à quels pays appartiennent ces NAs.

In [41]:
# Affichage des lignes corespondant aux valeurs manquantes

dispo_pop[dispo_pop['Proportion protéines animales (%)'].isnull()]

,Zone,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Proportion protéines animales (%),Taux évolution moyen population (%),PIB moyen/habitant ($),Quantité moyenne poulets importée ($)
20,Brunéi Darussalam,0.0,0.0,NaN,1.22,34671.88,843600.0


In [42]:
dispo_pop[dispo_pop['Quantité moyenne poulets importée ($)'].isnull()]

,Zone,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Proportion protéines animales (%),Taux évolution moyen population (%),PIB moyen/habitant ($),Quantité moyenne poulets importée ($)
42,Djibouti,2677.0,66.43,19.33,1.68,2640.69,NaN


Nous allons prendre le parti de supprimer le Brunéi Darussalam (sa suppression ne devrait pas avoir d'impact significatif sur l'analyse) et de ramplacer la valeur manquante de Djibouti par la valeurs moyenne des pays semblables (PIB moyen par habitant similaire).

In [43]:
# Suppression du Brunéi Darussalam

dispo_pop = dispo_pop[dispo_pop['Zone'] != 'Brunéi Darussalam']

In [44]:
dispo_pop[(dispo_pop['PIB moyen/habitant ($)'] >= 30000) & (dispo_pop['PIB moyen/habitant ($)'] <= 40000)]

,Zone,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Proportion protéines animales (%),Taux évolution moyen population (%),PIB moyen/habitant ($),Quantité moyenne poulets importée ($)
12,Bahamas,2043.0,61.37,68.44,0.98,30906.24,499000.0
50,France,3556.0,112.08,62.15,0.37,39580.86,24320200.0
71,Israël,3507.0,122.62,59.17,1.60,39534.03,6277200.0
72,Italie,3522.0,106.69,53.45,0.21,32978.49,13641800.0
74,Japon,2699.0,86.52,55.54,-0.16,38279.03,8011600.0
80,Koweït,3445.0,99.69,47.56,3.56,36147.48,5275000.0
107,Nouvelle-Calédonie,2750.0,83.12,58.08,1.20,35039.52,38000.0


In [45]:
qpi = dispo_pop[(dispo_pop['PIB moyen/habitant ($)'] >= 2000) & (dispo_pop['PIB moyen/habitant ($)'] <= 3000)]['Quantité moyenne poulets importée ($)'].mean()

In [46]:
# Remplacement du NA par la moyenne des pays ayant un PIB/habitant similaire

dispo_pop['Quantité moyenne poulets importée ($)'].fillna(qpi, inplace=True)

In [47]:
# Nouvelle recherche de valeurs manquantes dans dispo_pop

print('Valeurs manquantes dans le dataframe : ' + str(dispo_pop.isna().sum()))

Valeurs manquantes dans le dataframe : Zone                                                        0
Disponibilité alimentaire (Kcal/personne/jour)              0
Disponibilité de protéines en quantité (g/personne/jour)    0
Proportion protéines animales (%)                           0
Taux évolution moyen population (%)                         0
PIB moyen/habitant ($)                                      0
Quantité moyenne poulets importée ($)                       0
dtype: int64


### Chargement du fichier contenant les pays associés à leurs régions

In [48]:
# Import de PaysRégions

regions = pd.read_csv('PaysRégions.csv', on_bad_lines='skip')
regions.head()

,Région,Zone
0,Méditerranée orientale,Afghanistan
1,Afrique,Afrique du Sud
2,Europe,Albanie
3,Afrique,Algérie
4,Europe - UE,Allemagne


Nous pouvons maintenant joindre les régions à notre dataframe global. Afin de faire des régions une variable numérique, nous allons utiliser la fonction get_dummies.

In [49]:
# Jointure entre regions et dispo_pop

dispo_pop = pd.merge(dispo_pop, regions, on='Zone')
dispo_pop.head()

,Zone,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Proportion protéines animales (%),Taux évolution moyen population (%),PIB moyen/habitant ($),Quantité moyenne poulets importée ($),Région
0,Afghanistan,1997.0,54.09,19.52,2.98,553.52,9135200.0,Méditerranée orientale
1,Afrique du Sud,2987.0,83.36,41.46,1.51,6076.57,7425600.0,Afrique
2,Albanie,3400.0,119.50,55.50,-0.17,4306.02,5066400.0,Europe
3,Algérie,3345.0,92.85,27.68,2.06,4645.34,21107000.0,Afrique
4,Allemagne,3559.0,104.07,60.61,0.45,44308.71,173673000.0,Europe - UE


In [50]:
dispo_pop['Région'].unique()

array(['Méditerranée orientale', 'Afrique', 'Europe', 'Europe - UE',
       'Amérique du nord', 'Amérique du sud', 'Asie du Sud',
       'Pacifique occidental'], dtype=object)

In [51]:
dispo_pop = pd.get_dummies(dispo_pop, columns=['Région'])
dispo_pop.head()

,Zone,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Proportion protéines animales (%),Taux évolution moyen population (%),PIB moyen/habitant ($),Quantité moyenne poulets importée ($),Région_Afrique,Région_Amérique du nord,Région_Amérique du sud,Région_Asie du Sud,Région_Europe,Région_Europe - UE,Région_Méditerranée orientale,Région_Pacifique occidental
0,Afghanistan,1997.0,54.09,19.52,2.98,553.52,9135200.0,0,0,0,0,0,0,1,0
1,Afrique du Sud,2987.0,83.36,41.46,1.51,6076.57,7425600.0,1,0,0,0,0,0,0,0
2,Albanie,3400.0,119.50,55.50,-0.17,4306.02,5066400.0,0,0,0,0,1,0,0,0
3,Algérie,3345.0,92.85,27.68,2.06,4645.34,21107000.0,1,0,0,0,0,0,0,0
4,Allemagne,3559.0,104.07,60.61,0.45,44308.71,173673000.0,0,0,0,0,0,1,0,0


In [52]:
dispo_pop.describe()

,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Proportion protéines animales (%),Taux évolution moyen population (%),PIB moyen/habitant ($),Quantité moyenne poulets importée ($),Région_Afrique,Région_Amérique du nord,Région_Amérique du sud,Région_Asie du Sud,Région_Europe,Région_Europe - UE,Région_Méditerranée orientale,Région_Pacifique occidental
count,166.000000,166.000000,166.000000,166.000000,166.000000,1.660000e+02,166.000000,166.000000,166.000000,166.000000,166.000000,166.000000,166.000000,166.000000
mean,2860.475904,81.183916,42.737651,1.332711,13876.178072,1.345679e+07,0.240964,0.126506,0.078313,0.054217,0.138554,0.162651,0.096386,0.102410
std,454.719743,20.148292,16.039235,1.204405,19206.714725,5.408410e+07,0.428962,0.333425,0.269477,0.227130,0.346526,0.370163,0.296012,0.304104
min,1754.000000,39.910000,8.910000,-1.380000,406.860000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2515.500000,64.315000,28.927500,0.457500,2021.692500,5.276500e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2867.500000,81.150000,43.735000,1.200000,6093.015000,3.023500e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,3250.750000,96.067500,56.892500,2.200000,17190.962500,7.960900e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,3770.000000,141.340000,73.330000,5.520000,113672.390000,6.149210e+08,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [53]:
dispo_pop.to_csv('data_preprocessed.csv')